In [3]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle, FancyArrowPatch

class TreeNode:
    def __init__(self, coef = 1, zero = None, one = None, prev = None):
        self.coef = coef
        self.zero = None
        self.one = None
        self.prev = None
        
    def __str__(self):
        return str(self.coef)
    
    def __repr__(self):
        return self.__str__()
    
    def copy(self):
        if self.zero and self.one:
            return TreeNode(self.coef, self.zero.copy(), self.one.copy(), self.prev)
        elif self.zero:
            return TreeNode(self.coef, self.zero.copy(), None, self.prev)
        elif self.one:
            return TreeNode(self.coef, None, self.one.copy(), self.prev)
        else:
            return TreeNode(self.coef, None, None, self.prev)
        
class Tree:
    def __init__(self):
        self.root = TreeNode()
        self.dim = 0
    
    def add_layer(self):
        # add a TreeNode to the end of the zero branch
        current = self.root
        while current.zero:
            current = current.zero
        current.zero = TreeNode(prev = current)
        self.dim += 1
        
    def get_layer(self, layer):
        # return all the nodes in the layer
        if layer == 0:
            return [self.root]
        else:
            return self._get_layer(self.root, layer)
        
    def _get_layer(self, node, layer):
        if layer == 0:
            return [node]
        else:
            result = []
            if node.zero:
                result += self._get_layer(node.zero, layer-1)
            else:
                result += [None] * 2 ** (layer - 1)
            if node.one:
                result += self._get_layer(node.one, layer-1)
            else:
                result += [None] * 2 ** (layer - 1)
            return result
        
    def _remove_zero(self, level):
        # check if a layer has zeroes and replace them with None
        layer = self.get_layer(level-1)
        for node in layer:
            if node.one.coef == 0:
                node.one = None
        
    def _draw_tree(self, node, x, y, dx, dy, radius):
        if node:
            circle = Circle((x, y), radius, edgecolor='black', facecolor='white')
            plt.gca().add_patch(circle)
            plt.text(x, y, str(np.around(node.coef, decimals = 3)), fontsize=12, ha='center', va='center')
            if node.zero:
                child_x = x - dx
                child_y = y - dy
                plt.gca().add_patch(FancyArrowPatch((x - radius, y - radius), (child_x + radius, child_y + radius), mutation_scale=15, arrowstyle='-|>', color='blue'))
                self._draw_tree(node.zero, child_x, child_y, dx/2, dy, radius)
            if node.one:
                child_x = x + dx
                child_y = y - dy
                plt.gca().add_patch(FancyArrowPatch((x + radius, y - radius), (child_x - radius, child_y + radius), mutation_scale=15, arrowstyle='-|>', color='blue'))
                self._draw_tree(node.one, child_x, child_y, dx/2, dy, radius)
            
class Circuit(Tree):
    def __init__(self, dim):
        super().__init__()
        for i in range(dim):
            self.add_layer()
    
    def H(self, qubit):
        # apply Hadamard gate to qubit
        layer = self.get_layer(qubit-1)
        for node in layer:
            if node.one:
                node.zero.coef, node.one.coef = (node.zero.coef + node.one.coef)/np.sqrt(2), (node.zero.coef - node.one.coef)/np.sqrt(2)
            else:
                node.zero.coef = 1/np.sqrt(2)
                node.one = node.zero.copy()
        self._remove_zero(qubit)
                
    def X(self, qubit):
        # apply X gate to qubit
        layer = self.get_layer(qubit-1)
        for node in layer:
            node.zero.coef, node.one.coef = node.one.coef, node.zero.coef
        self._remove_zero(qubit)
            
    def Z(self, qubit):
        # apply Z gate to qubit
        layer = self.get_layer(qubit-1)
        for node in layer:
            node.one.coef *= -1
            
    def cx(self, control, target):
        # apply CNOT gate to control and target
        assert control == target - 1 # control and target must be adjacent
        layer = self.get_layer(control-1)
        for node in layer:
            if node.one:
                node.one.zero.coef, node.one.one.coef = node.one.one.coef, node.one.zero.coef
        self._remove_zero(target)
    
    def cz(self, control, target):
        # apply CZ gate to control and target
        assert control == target - 1 # control and target must be adjacent
        layer = self.get_layer(control-1)
        for node in layer:
            if node.one:
                node.one.one.coef *= -1
                
    def get_state(self, qubit):
        # return the state of the circuit as a dictionary
        state = {}
        i = qubit
        layer = self.get_layer(i)
        for j in range(len(layer)):
            if layer[j]:
                state[np.binary_repr(j).zfill(self.dim)] = layer[j].coef
            else:
                state[np.binary_repr(j).zfill(self.dim)] = 0
        return state
    
    def get_states(self):
        state = {}
        for i in range(self.dim):
            layer = self.get_layer(i)
            for j in range(len(layer)):
                if layer[j]:
                    state[np.binary_repr(j).zfill(i)] = layer[j].coef
                else:
                    state[np.binary_repr(j).zfill(i)] = 0
        return state
    
    def get_amplitude(self, qubit):
        return None
                
    def probs(self, qubit):
        # return the probability of measuring 0 and 1 on qubit
        if qubit == 0:
            return [1]
        else:
            prev_probs = self.probs(qubit-1)
            layer = self.get_layer(qubit)
            probs = np.array([])
            for q in range(len(prev_probs)):
                if layer[2*q]:
                    prob_0 = prev_probs[q] * np.abs(layer[2*q].coef) ** 2
                else:
                    prob_0 = 0
                if layer[2*q+1]:
                    prob_1 = prev_probs[q] * np.abs(layer[2*q+1].coef) ** 2
                else:
                    prob_1 = 0
                probs = np.append(probs, [prob_0, prob_1])
            return probs
        
    def create_equal_superposition(self):
        # apply Hadamard gate to all qubits
        for i in range(self.dim):
            self.H(i+1)
            
    def __str__(self):
        return str(self.get_states())
    
    def __repr__(self):
        return self.__str__()
    
    def draw_tree(self):
        plt.figure(figsize=(self.dim * 4, self.dim*2))
        self._draw_tree(self.root, 0.5, 0.9, 0.5, 0.1, 0.05)
        plt.xlim(0, 1)
        plt.ylim(0, 1)
        plt.axis('off')
        plt.show()

In [4]:
qc = Circuit(3)
print(qc)
print(qc.get_state())
qc.create_equal_superposition()
print(qc)
qc.cz(1, 2)
print(qc)
qc.get_state()

{'0': 1, '1': 0, '00': 1, '01': 0, '10': 0, '11': 0}


TypeError: Circuit.get_state() missing 1 required positional argument: 'qubit'

In [5]:
qc = Circuit(3)
print(qc)
print(qc.root.zero.copy().prev)

{'0': 1, '1': 0, '00': 1, '01': 0, '10': 0, '11': 0}
None
